In [32]:
import pandas as pd
import numpy as np
import random
import time

In [65]:
df = pd.read_excel('/home/admin-/Download/centre_list.xlsx')

In [66]:
'''
박민수	investment	사업부장	상무보	9/1 발령	ms.park@gsshop.com	02-2007-9789	010-6789-2923	m
배재성	investment	팀장		9/1 발령	jsbae@gsshop.com	02-2007-2419	010-0400-9238	m
최형민	investment				hm.choi@gsshop.com	02-2007-4451	010-3219-8925	m
'''

new_members = [{"name": "박민수", "department": "investment", "job": "사업부장", "position": "상무보", "etc": "9/1 발령", "email": "ms.park@gsshop.com", "tel": "02-2007-9789", "mobile": "010-6789-2923", "sex": "m"}
,{"name": "배재성", "department": "investment", "job": "팀장", "position": None, "etc": "9/1 발령", "email": "jsbae@gsshop.com", "tel": "02-2007-2419", "mobile": "010-0400-9238", "sex": "m"}
,{"name": "최형민", "department": "investment", "job": None, "position": None, "etc": "9/1 발령", "email": "hm.choi@gsshop.com", "tel": "02-2007-4451", "mobile": "010-3219-8925", "sex": "m"}]

df = df.append(new_members, ignore_index=True)


In [67]:
df = df.drop(df[df.name == "김준연"].index)

In [68]:
df.index = df.name.tolist()
df = df.drop(['name'], axis=1)

In [69]:
def make_tel_num(x):
    return '02-2007-' + str(random.randint(0, 9999)).zfill(4)

def make_mobile_num(x):
    return '010-' + str(random.randint(0, 9999)).zfill(4) + '-' + str(random.randint(0, 9999)).zfill(4)


df['tel'] = df.tel.apply(make_tel_num)
df['mobile']= df.mobile.apply(make_mobile_num)

In [70]:
df['executive_member'] = ~df.position.isna()

In [73]:
df.columns

Index(['department', 'job', 'position', 'etc', 'email', 'tel', 'mobile', 'sex',
       'executive_member'],
      dtype='object')

In [74]:
df = df[['department', 'sex', 'executive_member']]

In [96]:
df.groupby([df.sex, df.department]).size().to_frame()

In [76]:
member_list = df.index.tolist()
np.random.shuffle(member_list)


In [77]:
executives_list = df[df.executive_member].index.tolist()
np.random.shuffle(executives_list)
# pr_queue = df.dropna(subset=['position']).index

In [78]:
for x in executives_list:
    member_list.remove(x)

In [79]:
women_list = df.loc[df.sex == 'f'].index.tolist()
np.random.shuffle(women_list)

for x in women_list:
    member_list.remove(x)

In [81]:
team_list = []

In [82]:
def condition(df, cand1, cand2):
    if df.loc[cand1, 'department'] != df.loc[cand2, 'department']: 
        return True

In [82]:
for executive in executives_list:
    while True:
        woman = np.random.choice(women_list)

        if condition(df, executive, woman):
            team_list.append((executive, woman))
            women_list.remove(woman)
            break
        else:
            continue            

In [85]:
while True:
    # left_women = len(df.loc[df.sex == 'f'])
    if not women_list:
        break
        
    woman = np.random.choice(women_list)        
    women_list.remove(woman)

    while True:
        cand = np.random.choice(member_list)
        
        if condition(df, woman, cand):
            team_list.append((woman, cand))
            member_list.remove(cand)
            break
        else:
            continue 

In [88]:
left_depts = df.loc[member_list, 'department'].unique()

for dept in left_depts:
    left_dept = df.loc[(df.department == dept) & (df.index.isin(member_list))]    
    
    if len(left_dept) > len(member_list)/2:
        print("no answer! restart the programme")

In [90]:
origin_list = member_list[:]
temp_list = []

while True:
    left_depts = df.loc[member_list, 'department'].unique()

    if not member_list:
        print("member_list is empty.")
        print("team matching is done.")
        team_list.extend(temp_list)
        break
    elif len(left_depts)== 1:
        print("no answer. team matching has started again.")
        temp_list = []
        member_list = origin_list[:]
                
    cand1 = np.random.choice(member_list)        
    member_list.remove(cand1)

    while True:
        cand2 = np.random.choice(member_list)
        
        if condition(df, cand1, cand2):
            temp_list.append((cand1, cand2))
            member_list.remove(cand2)
            break
        else:
            continue 

no answer. team matching has started again.
no answer. team matching has started again.
no answer. team matching has started again.
no answer. team matching has started again.
member_list is empty.
team matching is done.


In [93]:
df_team = pd.DataFrame(data)

In [94]:
df_team['mem1_name'] = [tup[0] for tup in team_list]
df_team['mem1_depart'] = [df.loc[tup[0]].department for tup in team_list]
# df_team['mem1_sex'] = [df.loc[tup[0]].sex.values[0] for tup in team_list]

df_team['mem2_name'] = [tup[1] for tup in team_list]
df_team['mem2_depart'] = [df.loc[tup[1]].department for tup in team_list]
# df_team['mem2_sex'] = [df.loc[tup[1]].sex.values[0] for tup in team_list]


In [123]:
df_team.to_excel('/home/admin-/Download/team_list_2.xlsx', index=False)